# Modeling with Gurobi

In this tutorial we will learn the basics about building and solving optimization models using the Python 
interface of Gurobi.
In order to use Gurobi in your Python application it is necessary that your Python application knows where to find the Gurobi files. If you are in doubt read the instructions on Absalon for setting up Gurobi in your virtual environment. Once Gurobi has been added to the virtual environment we are ready to write a Python application that uses Gurobi.

In this tutorial we will build the following optimization model.
$$\max  x +   y + 2 z$$
$$ \text{s.t.~} x + 2 y + 3 z \leq 4$$
$$ x +   y       \geq 1 $$
$$ x  \in\{0,1\} $$
$$ y, z \geq 0$$


To use Gurobi in a Python file we start with the following statement which gives our file access to the classes of the `gurobipy` package.

In [1]:
from gurobipy import *

The first step consists of building an object of the class `Model`. This represents a container for the optimization problem we will build. To this model we will add variables, constraints and objective function.

In [2]:
# Create a new model
m = Model("example_model")
print(m)

Academic license - for non-commercial use only - expires 2022-08-12
Using license file /Library/gurobi912/gurobi.lic
<gurobi.Model Continuous instance example_model: 0 constrs, 0 vars, No parameter changes>


`Model` is a Python class designed by Gurobi and `m` is an object, an instance, of the `Model` class (have a look at the [Python tutorial](./python_tutorial.ipynb) if you are unsure what a class is). As any class, it has a constructor (which we have just used to build the model `m`) and a number of method for interacting with objects. 
[The Python API details](https://www.gurobi.com/documentation/9.0/refman/py_model.html) page describes the class `Model` in greater details. **You should bookmark this page and familiarize with it as it will be your go-to reference when you are in doubt or when you want to learn more**. In what follows you should go back and forth between this tutorial and the Gurobi Python API to read the details about the methods used. Links to the methods will be provided.

## Adding Variables

Now that our model is created, we can use its methods to create the optimization problem, starting from the decision variables $x$, $y$ and $z$. We add variables using the [method `addVar()`](https://www.gurobi.com/documentation/9.0/refman/py_model_addvar.html). At the Python API you will see that this method accepts the following arguments:

`addVar (lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="", column=None ) `

Let us create the variable $x$, which is binary.

In [3]:
# Adding variable x
x = m.addVar(vtype=GRB.BINARY,name = "x")
print(m)
print(x)

<gurobi.Model Continuous instance example_model: 0 constrs, 0 vars, No parameter changes>
<gurobi.Var *Awaiting Model Update*>


Note two things. First, the model does not yet count the variable. We could call the method [`update`](https://www.gurobi.com/documentation/9.0/refman/py_model_update.html) or wait to add the objective and constraints, as those will automatically update the model. Second, the `addVar` method returns an object of the Gurobi [`Var` class](https://www.gurobi.com/documentation/9.0/refman/py_var.html) (the object `x` in our code) which represents decision variables and has its own methods and fiels. Let us add the other variables. 

In [4]:
y = m.addVar(name = "y")
z = m.addVar(lb = 0.0, ub = GRB.INFINITY,vtype= GRB.CONTINUOUS, name = "z")
print(m)
print(y)
print(z)

<gurobi.Model Continuous instance example_model: 0 constrs, 0 vars, No parameter changes>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>


Notice that both $y$ and $z$ are non-negative continuous variables, but they have been added in two different ways. This was an arbitrary choice, both methods produce the same result, since the arguments `lb`, `ub`, `vtype` have default to $0$, $\infty$ and `GRB.CONTINUOUS`, respectively. The same result would be achieved passing any subset of these arguments. Finally, not that if we had to create multidimensional variables, the `Model` class has  [method addVars](https://www.gurobi.com/documentation/9.0/refman/py_model_addvars.html). We will get to that in a later tutorial. 

## Adding the Objective function

Let us now add the objective function. We use the [method setObjective()](https://www.gurobi.com/documentation/9.0/refman/py_model_setobjective.html) which is specified as follows:

`setObjective ( expr, sense=None ) `

It taks as argument a linear expression, that is an object of the [class `LinExpr`](https://www.gurobi.com/documentation/9.0/refman/py_lex.html) and a sense, which can be either `GRB.MINIMIZE` or `GRB.MAXIMIZE`. Objects of the class `LinExpr` represent linear expressions of the decision variables. They are very easy to create: we just do operations with the decision variables. 

In [5]:
expr = x + y + 2 *z

Alternatively we could write the same code as follows

In [6]:
expr = 0 
expr += x
expr += y
expr += 2 * z

The we pass the objective to the optimization model with the instruction to maximize it

In [7]:
m.setObjective(expr,GRB.MAXIMIZE)
print(m)

<gurobi.Model Continuous instance example_model: 0 constrs, 0 vars, No parameter changes>


Notice that if we call the method `update()` we see that the model has three decision variables.

In [8]:
m.update()
print(m)

<gurobi.Model MIP instance example_model: 0 constrs, 3 vars, No parameter changes>


## Adding Constraints

To add constraints we proceed similarly to what we have seen for the objective function. First, we create a linear expression and then we pass it to the [method addConstr()](https://www.gurobi.com/documentation/9.0/refman/py_model_addconstr.html). The method is specified as follows

`addConstr ( lhs, sense=None, rhs=None, name="" ) `

it takes a left-hand-side and a right-hand side expression (again objects of the [class LinExpr](https://www.gurobi.com/documentation/9.0/refman/py_lex.html)), a sense among `GRB.LESS_EQUAL`, `GRB.EQUAL`, or `GRB.GREATER_EQUAL`, and a name. Nevertheless, only the `lhs` expression is necessary. In fact, we can use the operators `==`, `<= ` and `>=` to create constraints. We will use both ways in this example. Let us create the first constraint

In [9]:
expr = x + 2 * y + 3* z
m.addConstr(expr, GRB.LESS_EQUAL,4,'c1')
m.update()
print(m)

<gurobi.Model MIP instance example_model: 1 constrs, 3 vars, No parameter changes>


Let us add the second constraint in a more compact way

In [10]:
m.addConstr( x+y >= 1,'c2')
m.update()

A convenience method is the [method `write()`](https://www.gurobi.com/documentation/9.0/refman/py_model_write.html) of the `Model` class. It can print the model in a human readable way and help us understand whether we made any mistakes.

In [13]:
m.write("model.lp")

The code above creates a file named "model.lp" which looks like the following

\ Model example_model
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  x + y + 2 z
Subject To
 c1: x + 2 y + 3 z <= 4
 c2: x + y >= 1
Bounds
Binaries
 x
End


## Solving the problem and accessing the solution

Finally we can solve our optimization problem 

In [14]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xcdfdeb50
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.5000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 2.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


The `Model` class and `Var` class have several methods to access solution information. 
For example, we can access the optimal objective value as follows

In [15]:
print('Objective value: %g' % m.objVal)

Objective value: 3


and the optimal solution using the `.x` field of `Var` objects which stores the optimal value of a decision variable.

In [16]:
print('%s %g' % (x.varName, x.x))
print('%s %g' % (y.varName, y.x))
print('%s %g' % (z.varName, z.x))

x 1
y 0
z 1
